In [2]:
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tick

In [3]:
def make_and_save_api_call(endpoint, params, filename):
    call = requests.get(endpoint.format(**params))
    response = call.json()
    with open(filename, "w", encoding="utf-8") as file:
        json.dump(response, file)

In [3]:
games_endpoint = "https://api.collegefootballdata.com/games?year={year}"

In [11]:
make_and_save_api_call(games_endpoint, {"year": "2012"}, "games_endpoint_2012.json")
make_and_save_api_call(games_endpoint, {"year": "2013"}, "games_endpoint_2013.json")
make_and_save_api_call(games_endpoint, {"year": "2014"}, "games_endpoint_2014.json")
make_and_save_api_call(games_endpoint, {"year": "2015"}, "games_endpoint_2015.json")
make_and_save_api_call(games_endpoint, {"year": "2016"}, "games_endpoint_2016.json")
make_and_save_api_call(games_endpoint, {"year": "2017"}, "games_endpoint_2017.json")
make_and_save_api_call(games_endpoint, {"year": "2018"}, "games_endpoint_2018.json")

game data does not store attendance data before 2012

In [55]:
games_2012 = pd.read_json("games_endpoint_2012.json")
games_2013 = pd.read_json("games_endpoint_2013.json")
games_2014 = pd.read_json("games_endpoint_2014.json")
games_2015 = pd.read_json("games_endpoint_2015.json")
games_2016 = pd.read_json("games_endpoint_2016.json")
games_2017 = pd.read_json("games_endpoint_2017.json")
games_2018 = pd.read_json("games_endpoint_2018.json")

In [56]:
all_games = pd.concat([games_2012, games_2013, games_2014, games_2015, games_2016, games_2017, games_2018], ignore_index=True)
all_games.head()

,attendance,away_conference,away_line_scores,away_points,away_team,conference_game,home_conference,home_line_scores,home_points,home_team,id,neutral_site,season,season_type,start_date,venue,venue_id,week
0,15250.0,None,"[10, 14, 3, 0]",27,Southeast Missouri State,False,Mid-American,"[10, 14, 7, 7]",38,Central Michigan,322432117,False,2012,regular,2012-08-30T19:00:00.000Z,Kelly/Shorts Stadium,3786.0,1
1,15121.0,None,"[0, 7, 0, 14]",21,Towson,False,Mid-American,"[17, 10, 7, 7]",41,Kent State,322432309,False,2012,regular,2012-08-30T19:00:00.000Z,Dix Stadium,3696.0,1
2,12616.0,Conference USA,"[7, 28, 14, 7]",56,UCF,False,Mid-American,"[0, 0, 7, 7]",14,Akron,322432006,False,2012,regular,2012-08-30T19:00:00.000Z,Summa Field at InfoCision Stadium,3768.0,1
3,12725.0,Mid-American,"[0, 13, 0, 13]",26,Eastern Michigan,True,Mid-American,"[10, 3, 21, 3]",37,Ball State,322432050,False,2012,regular,2012-08-30T19:00:00.000Z,Scheumann Stadium,3919.0,1
4,38393.0,SEC,"[7, 3, 0, 7]",17,South Carolina,True,SEC,"[0, 10, 3, 0]",13,Vanderbilt,322430238,False,2012,regular,2012-08-30T19:00:00.000Z,Vanderbilt Stadium,3973.0,1


In [57]:
all_games[all_games["attendance"] == 0]

,attendance,away_conference,away_line_scores,away_points,away_team,conference_game,home_conference,home_line_scores,home_points,home_team,id,neutral_site,season,season_type,start_date,venue,venue_id,week
388,0.0,FBS Independents,"[14, 3, 14, 0]",31,Navy,False,Mid-American,"[3, 7, 3, 0]",13,Central Michigan,322862117,False,2012,regular,2012-10-12T20:00:00.000Z,Kelly/Shorts Stadium,3786.0,7
571,0.0,ACC,"[14, 2, 7, 10]",33,Virginia,True,ACC,"[0, 0, 0, 6]",6,NC State,323080152,False,2012,regular,2012-11-03T12:30:00.000Z,Carter-Finley Stadium,3670.0,10
573,0.0,Mid-American,"[0, 14, 0, 28]",42,Western Michigan,True,Mid-American,"[6, 7, 10, 8]",31,Central Michigan,323082117,False,2012,regular,2012-11-03T13:00:00.000Z,Kelly/Shorts Stadium,3786.0,10
678,0.0,Big East,"[14, 14, 14, 21]",63,Temple,False,FBS Independents,"[0, 10, 14, 8]",32,Army,400368353,False,2012,regular,2012-11-17T12:00:00.000Z,Blaik Field at Michie Stadium,3841.0,12
843,0.0,ACC,"[0, 3, 7, 7]",17,Syracuse,False,Big Ten,"[0, 6, 7, 10]",23,Penn State,332430213,True,2013,regular,2013-08-31T15:30:00.000Z,None,NaN,1
1097,0.0,American Athletic,"[3, 9, 0, 0]",12,Connecticut,False,Mid-American,"[14, 14, 6, 7]",41,Buffalo,332712084,False,2013,regular,2013-09-28T15:30:00.000Z,UB Stadium,3965.0,5
1197,0.0,Mid-American,"[7, 3, 3, 13]",26,Central Michigan,True,Mid-American,"[3, 0, 7, 13]",23,Ohio,332850195,False,2013,regular,2013-10-12T14:00:00.000Z,Peden Stadium,3876.0,7
1207,0.0,Sun Belt,"[7, 7, 7, 14]",35,Troy,True,Sun Belt,"[7, 0, 14, 7]",28,Georgia State,332852247,False,2013,regular,2013-10-12T15:30:00.000Z,Georgia Dome,3495.0,7
1462,0.0,Mid-American,"[0, 7, 0, 7]",14,Akron,True,Mid-American,"[3, 3, 7, 0]",13,UMass,333200113,False,2013,regular,2013-11-16T13:00:00.000Z,Gillette Stadium,3738.0,12
1500,0.0,None,"[13, 0, 7, 0]",20,Old Dominion,False,ACC,"[14, 35, 31, 0]",80,North Carolina,333270153,False,2013,regular,2013-11-23T12:00:00.000Z,Kenan Memorial Stadium,3787.0,13


In [58]:
all_games = all_games[all_games["attendance"] > 0]

In [59]:
venues_endpoint = "https://api.collegefootballdata.com/venues"
make_and_save_api_call(venues_endpoint, {}, "venues_endpoint.json")
venues = pd.read_json("venues_endpoint.json")
venues

,capacity,city,country_code,dome,elevation,grass,id,location,name,state,year_constructed,zip
0,10000,Tuskegee,US,0.0,122.800000,NaN,4747,None,Abbott Memorial Alumni Stadium,AL,1925.0,NaN
1,28500,Baton Rouge,US,0.0,20.242132,NaN,3599,"{'x': 30.5221461, 'y': -91.1896087}",Ace W. Mumford Stadium,LA,1928.0,NaN
2,30343,Las Cruces,US,0.0,1208.201294,0.0,3601,"{'x': 32.2796202, 'y': -106.7411148}",Aggie Memorial Stadium,NM,1978.0,88003.0
3,21500,Greensboro,US,0.0,235.122452,NaN,3603,"{'x': 36.0814337, 'y': -79.7700391}",Aggie Stadium,NC,1981.0,NaN
4,10849,Davis,US,0.0,11.382592,NaN,3602,"{'x': 38.5365266, 'y': -121.7627936}",Aggie Stadium,CA,2007.0,NaN
5,65000,San Antonio,US,0.0,201.552689,0.0,3604,"{'x': 29.4169834, 'y': -98.4788143}",Alamodome,TX,1993.0,78299.0
6,36387,Boise,US,0.0,823.222107,0.0,3653,"{'x': 43.6028839, 'y': -116.1958882}",Albertsons Stadium,ID,1970.0,83725.0
7,12283,Grand Forks,US,0.0,255.247208,0.0,3605,"{'x': 47.910622, 'y': -97.090996}",Alerus Center,ND,2001.0,NaN
8,11075,San Luis Obispo,US,0.0,84.683243,NaN,1027,"{'x': 35.2984123, 'y': -120.6650005}",Alex G. Spanos Stadium,CA,2006.0,NaN
9,10000,Orono,US,0.0,35.481754,0.0,3607,"{'x': 44.905128, 'y': -68.6724718}",Alfond Stadium,ME,1998.0,4469.0


join on venue id:

In [60]:
venues = venues.rename(columns={"id": "venue_id"})


In [61]:
game_and_venue_data = pd.merge(all_games, venues, on="venue_id")

In [62]:
game_and_venue_data.head()

,attendance,away_conference,away_line_scores,away_points,away_team,conference_game,home_conference,home_line_scores,home_points,home_team,...,city,country_code,dome,elevation,grass,location,name,state,year_constructed,zip
0,15250.0,None,"[10, 14, 3, 0]",27,Southeast Missouri State,False,Mid-American,"[10, 14, 7, 7]",38,Central Michigan,...,Mount Pleasant,US,0.0,240.900772,0.0,"{'x': 43.577731299999996, 'y': -84.7709904}",Kelly/Shorts Stadium,MI,1972.0,48859.0
1,35127.0,Big Ten,"[7, 17, 10, 7]",41,Michigan State,False,Mid-American,"[0, 0, 0, 7]",7,Central Michigan,...,Mount Pleasant,US,0.0,240.900772,0.0,"{'x': 43.577731299999996, 'y': -84.7709904}",Kelly/Shorts Stadium,MI,1972.0,48859.0
2,14081.0,Mid-American,"[7, 17, 7, 10]",41,Ball State,True,Mid-American,"[0, 10, 7, 13]",30,Central Michigan,...,Mount Pleasant,US,0.0,240.900772,0.0,"{'x': 43.577731299999996, 'y': -84.7709904}",Kelly/Shorts Stadium,MI,1972.0,48859.0
3,10172.0,Mid-American,"[7, 0, 0, 7]",14,Akron,True,Mid-American,"[14, 0, 14, 7]",35,Central Michigan,...,Mount Pleasant,US,0.0,240.900772,0.0,"{'x': 43.577731299999996, 'y': -84.7709904}",Kelly/Shorts Stadium,MI,1972.0,48859.0
4,7223.0,Mid-American,"[7, 0, 3, 6]",16,Miami (OH),True,Mid-American,"[7, 14, 0, 9]",30,Central Michigan,...,Mount Pleasant,US,0.0,240.900772,0.0,"{'x': 43.577731299999996, 'y': -84.7709904}",Kelly/Shorts Stadium,MI,1972.0,48859.0


In [63]:
game_and_venue_data["attendance_percentage"] = game_and_venue_data["attendance"] / game_and_venue_data["capacity"]

In [70]:
game_and_venue_data.to_csv("processed_data/game_and_venue_data.csv")

In [4]:
rankings_endpoint = "https://api.collegefootballdata.com/rankings?year={year}"
make_and_save_api_call(rankings_endpoint, {"year": "2012"}, "rankings_endpoint_2012.json")
make_and_save_api_call(rankings_endpoint, {"year": "2013"}, "rankings_endpoint_2013.json")
make_and_save_api_call(rankings_endpoint, {"year": "2014"}, "rankings_endpoint_2014.json")
make_and_save_api_call(rankings_endpoint, {"year": "2015"}, "rankings_endpoint_2015.json")
make_and_save_api_call(rankings_endpoint, {"year": "2016"}, "rankings_endpoint_2016.json")
make_and_save_api_call(rankings_endpoint, {"year": "2017"}, "rankings_endpoint_2017.json")
make_and_save_api_call(rankings_endpoint, {"year": "2018"}, "rankings_endpoint_2018.json")

In [6]:
rankings_2012 = pd.read_json("rankings_endpoint_2012.json")
rankings_2013 = pd.read_json("rankings_endpoint_2013.json")
rankings_2014 = pd.read_json("rankings_endpoint_2014.json")
rankings_2015 = pd.read_json("rankings_endpoint_2015.json")
rankings_2016 = pd.read_json("rankings_endpoint_2016.json")
rankings_2017 = pd.read_json("rankings_endpoint_2017.json")
rankings_2018 = pd.read_json("rankings_endpoint_2018.json")

In [53]:
rankings_2012.head()

,polls,season,seasonType,week
0,"[{'poll': 'Coaches Poll', 'ranks': [{'rank': 2...",2012,regular,14
1,"[{'poll': 'Coaches Poll', 'ranks': [{'rank': 1...",2012,regular,13
2,"[{'poll': 'Coaches Poll', 'ranks': [{'rank': 2...",2012,regular,12
3,"[{'poll': 'Coaches Poll', 'ranks': [{'rank': 2...",2012,regular,11
4,"[{'poll': 'Coaches Poll', 'ranks': [{'rank': 2...",2012,regular,10


In [50]:
datasets = [rankings_2012, rankings_2013, rankings_2014, rankings_2015, rankings_2016, rankings_2017, rankings_2018]

ap_polls = []
years = []
weeks = []

for dataset in datasets:
    for index, row in dataset.iterrows():
        weeks.append(row.week)
        years.append(row.season)
        polls = row.polls
        for i in range(len(polls)):
            # get poll names
            if polls[i]["poll"] == "AP Top 25":
                #ap_polls.append(polls[i]["ranks"])
                ranked_teams = polls[i]["ranks"]
                schools = []
                for ranking in ranked_teams:
                    schools.append(ranking["school"])
                ap_polls.append(schools)

In [51]:
ap_ranking_data = pd.DataFrame()
ap_ranking_data["ap_poll"] = ap_polls 
ap_ranking_data["year"] = years
ap_ranking_data["week"] = weeks
ap_ranking_data.head()

,ap_poll,year,week
0,"[Utah State, Ohio State, Michigan, Nebraska, N...",2012,14
1,"[Rutgers, Louisville, Utah State, Ohio State, ...",2012,13
2,"[Rutgers, Louisiana Tech, Louisville, Ohio Sta...",2012,12
3,"[Rutgers, Louisiana Tech, Louisville, Ohio Sta...",2012,11
4,"[Louisiana Tech, Louisville, Ohio State, Nebra...",2012,10


In [ ]:
# get station name

In [109]:
weather_endpoint = "https://www.ncdc.noaa.gov/cdo-web/api/v2/dataset=daily-summaries&dataTypes=TMAX,PRCP,TMIN&stations={stations}&startDate={startDate}&endDate={endDate}&includeAttributes=true&format=json"

In [128]:
import requests


def get_noaa_data(url, data_type, header):
    r = requests.get(url, data_type, headers=header)
    print("response: ", r.content)


token = 'token_here'
creds = dict(token=token)
dtype = 'dataset'
weather_endpoint = "https://www.ncdc.noaa.gov/cdo-web/api/v2/stations"

get_noaa_data(weather_endpoint, dtype, creds)

response:  b'{"metadata":{"resultset":{"offset":1,"count":139164,"limit":25}},"results":[{"elevation":139,"mindate":"1948-01-01","maxdate":"2014-01-01","latitude":31.5702,"name":"ABBEVILLE, AL US","datacoverage":0.8813,"id":"COOP:010008","elevationUnit":"METERS","longitude":-85.2482},{"elevation":249.3,"mindate":"1938-01-01","maxdate":"2015-11-01","latitude":34.2553,"name":"ADDISON, AL US","datacoverage":0.5059,"id":"COOP:010063","elevationUnit":"METERS","longitude":-87.1814},{"elevation":302.1,"mindate":"1940-05-01","maxdate":"1962-03-01","latitude":34.41667,"name":"ADDISON CENTRAL TOWER, AL US","datacoverage":0.9658,"id":"COOP:010071","elevationUnit":"METERS","longitude":-87.31667},{"elevation":172.2,"mindate":"1995-04-01","maxdate":"2015-11-01","latitude":33.17833,"name":"ALABASTER SHELBY CO AIRPORT, AL US","datacoverage":0.8064,"id":"COOP:010116","elevationUnit":"METERS","longitude":-86.78167},{"elevation":183.8,"mindate":"1949-01-01","maxdate":"1949-12-01","latitude":34.6891,"name

In [86]:
weather_endpoint = "https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&dataTypes=TMAX,PRCP,TMIN&stations={stations}&startDate={startDate}&endDate={endDate}&includeAttributes=true&format=json"

In [112]:
make_and_save_api_call(weather_endpoint, {"stations": "USC00456096", "startDate": "2007-01-01", "endDate": "2007-01-02"}, "weather_test.json")

In [113]:
weather_test = pd.read_json("weather_test.json")
weather_test

ValueError: If using all scalar values, you must pass an index

In [ ]:
weather_endpoint = "https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets?lat={lat}&lon={lon}&product=time-series&begin=2004-01-01T00:00:00&end=2019-04-20T00:00:00&maxt=maxt&mint=mint"

In [50]:
for i, row in game_and_venue_data.iterrows():
    if row["location"]:
        lat = row["location"]["x"]
        lon = row["location"]["y"]
    zip_code = row["zip"]
    kickoff = row["start_date"]